In [ ]:
import torch
from torchvision.models import DenseNet
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import tables
import random
import cv2
import glob
import random
import pandas as pd
import re

In [ ]:
dataname="CUP010722"
gpuid=0

# --- densenet params
#these parameters get fed directly into the densenet class, and more description of them can be discovered there
num_classes=4    #number of classes in the data mask that we'll aim to predict
in_channels= 3  #input channel of the data, RGB = 3


growth_rate=32 
block_config=(2, 2, 2, 2)
num_init_features=64
bn_size=4
drop_rate=0.5
batch_size=128
patch_size=224
random.seed(7654788126099255772) 
print(torch.cuda.get_device_properties(gpuid))
torch.cuda.set_device(gpuid)
device = torch.device(f'cuda:{gpuid}' if torch.cuda.is_available() else 'cpu')

# phases = ["val"] #how many phases did we create databases for?
# validation_phases= ["val"] #when should we do valiation? note that validation is *very* time consuming, so as opposed to doing for both training and validation, we do it only for vlaidation at the end of the epoch
#                            #additionally, using simply [], will skip validation entirely, drastically speeding things up



In [ ]:
model = DenseNet(growth_rate=growth_rate, block_config=block_config,
                 num_init_features=num_init_features, 
                 bn_size=bn_size, 
                 drop_rate=drop_rate, 
                 num_classes=num_classes).to(device)
print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")

In [ ]:
class LayerActivations():
    features=None
    def __init__(self,layer):
        self.hook = layer.register_forward_hook(self.hook_fn)
    def hook_fn(self,module,input,output):
        self.features = output.cpu()
    def remove(self):
        self.hook.remove()



In [ ]:
checkpoint = torch.load(f"{dataname}_densenet_best_model.pth")
model.load_state_dict(checkpoint["model_dict"])



In [ ]:

#helper function for visualization
def plot_kernels(tensor, num_cols=8 ,cmap="gray"):
    if not len(tensor.shape)==4:
        raise Exception("assumes a 4D tensor")
#    if not tensor.shape[1]==3:
#        raise Exception("last dim needs to be 3 to plot")
    num_kernels = tensor.shape[0] * tensor.shape[1]
    num_rows = 1+ num_kernels // num_cols
    fig = plt.figure(figsize=(num_cols,num_rows))
    i=0
    t=tensor.data.numpy()
    for t1 in t:
        for t2 in t1:
            i+=1
            ax1 = fig.add_subplot(num_rows,num_cols,i)
            ax1.imshow(t2 , cmap=cmap)
            ax1.axis('off')
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])

    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()


In [ ]:
#Final block Layer 2 
#w = model.features.denseblock2.denselayer1.conv2
#final layer before classification
w = model.features.norm5


In [ ]:
data = model.classifier.weight.cpu().detach().numpy()
dfData = pd.DataFrame(np.absolute(data)).T
print(dfData)
dfData.to_csv('weight.csv',index=False)
criteria = dfData[0]>=0.09227
idxKept = criteria.index[criteria].T.tolist()

In [ ]:
dr=LayerActivations(w)


In [ ]:
fname = 'E:\\Esophagus\\DenseNetTrain224NormalAbnormal\\NormalBE\\867;1141_4033_9409.png'
img=cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
img
plt.imshow(img)

In [ ]:
#img2 = plt.imread('E:\\Esophagus\\DenseNetTrain224NormalAbnormal\\NormalBE\\867;1141_4033_9409.png')
#plt.imshow(img2)

In [ ]:
dr

In [ ]:
output=model(torch.from_numpy(np.transpose(img[None,::],(0,3,1,2))).to(device,dtype=torch.float))

In [ ]:
plot_kernels(dr.features,8,cmap="rainbow")

In [ ]:
feats = dr.features.detach().numpy()
feats = feats[:,idxKept,:,:]


In [ ]:
ooo = np.vstack((feats,feats))
print(ooo.shape)

In [ ]:
asdas = torch.flatten(torch.from_numpy(ooo[1,:]))
print(asdas.numpy().shape[0])

In [ ]:
#output=model(torch.from_numpy(np.transpose(img2[None,::],(0,3,1,2))).to(device,dtype=torch.float))
#plot_kernels(dr.features,8,cmap="rainbow")

In [ ]:
# abnormalTable = pd.read_excel('E:\\Esophagus\\Updated Worksheet for Imagingy.xlsx',
#                               sheet_name='Abnormal0422', usecols='A,F')
# normalTable = pd.read_excel('E:\\Esophagus\\Updated Worksheet for Imagingy.xlsx',
#                             sheet_name='NormaBE', usecols='B,C')
# abnormalTable = abnormalTable.dropna()
# normalTable = normalTable.dropna()
# dataTable = normalTable.append(abnormalTable, ignore_index=True)
# abnormalImageDir = glob.glob('E:\\Esophagus\\DenseNetTrain224NormalAbnormal\\AbnormalBE\\*.png')
normalImageDir = glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\OrganSiteClassifying\\**\\**\\*.png')
# normalImageDir.extend(abnormalImageDir)
imageDir = normalImageDir
feats = np.zeros((len(imageDir), 6272))
label = []
imgName = []

In [ ]:
normalImageDir

In [ ]:
from tqdm import tqdm_notebook
for i in tqdm_notebook(range(len(imageDir))):
    #image = cv2.cvtColor(cv2.imread(imageDir[i]),cv2.COLOR_BGR2RGB)
    image = plt.imread(imageDir[i])
    output=model(torch.from_numpy(np.transpose(image[None,::],(0,3,1,2))).to(device,dtype=torch.float))
    ff = dr.features.detach().numpy()
    feats[i,:] = np.reshape(ff[:,idxKept,:,:], [1, 6272 ])


In [ ]:
imgName

In [ ]:
np.savez_compressed('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\DLfeats\\dlFeats.npz',
                    Feats=feats,
                    Dir=normalImageDir)

In [ ]:
info = dir(model.features.norm5)

In [ ]:
info

In [ ]:
model.eval

In [ ]:
np.savez_compressed('E:\\Esophagus\\DenseNetTrain224NormalAbnormal\\umapFeatureImgName',
                    imgName=imgName)